In [1]:
import pandas as pd
import os

import spacy 
import subprocess
from string import punctuation

from rake_nltk import Rake

In [2]:
def rake_implement(x,r):
    r.extract_keywords_from_text(x)
    return r.get_ranked_phrases()

In [3]:
dir_path = os.path.dirname(os.path.realpath("__file__"))
df = pd.read_json(dir_path + "/data_6_24_seattle.json", orient="records")

In [4]:
df = df.dropna(subset=["details"])

In [5]:
df.head()

,companyName,date,details,rating,summary,title,url,location
0,Robert Half,30+ days ago,2-7 years of experience\nExpert in SQL queries...,3.9,2-7 years of experience.\nExpert in SQL querie...,Data Scientist,https://www.indeed.com/viewjob?jk=7e25d46b2b2f...,NaN
1,Microsoft,30+ days ago,Data Scientists at Microsoft help to improve t...,4.2,You will use data exploration techniques to di...,Full Time Opportunities for PhD Students or Re...,https://www.indeed.com/viewjob?jk=97ba8c5ac004...,"Redmond, WA 98052 (Overlake area)"
2,University of Washington,30+ days ago,Notes: 05/25/2021: This is a RE-POSTING. Candi...,4.1,"Bring together data, analytic engines, and dat...",DATA ANALYST (TEMPORARY),https://www.indeed.com/viewjob?jk=4cea00d8576c...,"Seattle, WA 98195 (University District area)"
3,"PitchBook Data, Inc.",30+ days ago,At PitchBook we work to provide global profess...,3.7,Strong communication and data presentation ski...,Associate Machine Learning Engineer,https://www.indeed.com/viewjob?jk=84cb60a7a416...,"Seattle, WA 98164 (Downtown area)"
4,NaN,6 days ago,About the Team\nData is at the core of Outreac...,NaN,The data velocity is in Terabytes.\nImplementi...,"Staff Data Scientist, Natural Language Processing",https://www.indeed.com/viewjob?jk=80b823b2023d...,"Seattle, WA"


In [6]:
df.shape

(160, 8)

In [7]:
df.details = df.details.astype(str)
df_nltk = df.copy()
r = Rake()
df_nltk["key_words"] = df_nltk["details"].apply(lambda x: rake_implement(x,r))
exploded = pd.concat([pd.Series(row['title'],  row['key_words']) for _, row in df_nltk.iterrows()]).reset_index()
exploded.columns = ["key_words", "title"]

In [98]:
lang_df = pd.read_csv(dir_path + "/code_languages.csv")
lang_df['name'] = lang_df['name']#.str.lower()
lang_list = lang_df.name.unique().tolist()

extra_skills = [
    "statistics",
    "machine learning",
    "NLP",
    "natural language processing",
    "time series",
    "big data",
    "data visualization",
    "Scikit",
    "scipy",
    "plotly",
    "apache",
    "Tensorflow",
    "stats",
    "Statistics",
    "Mathematics",
    "computer science",
    "data engineering",
    "Numpy",
    "Pandas",
    "AWS",
    "Windows",
    "Linux",
    "Excel",
    "deep learning",
    "neural network",
    "Data Engineer",
    "knn",
    "supervised machine learning",
    "unsupervised machine learning",
    "Python",
    "SQL",
    "Sql",
    "Spark",
    "Hadoop",
    "Elasticsearch",
    "Full-Stack",
    "C++",
    "C#",
    "C/",
    "/C",
    " C ",
    "/R",
    "R/",
    "R,",
    " R ",
    "Ruby",
    "Matplotlib",
    "ML",
    "Java",
    "Javascript",
    "PHP",
    "Php",
    "Html",
    "CSS",
    "JScript",
    "Database",
    "blockchain",
    "Assembly",
    "Apex",
    "Git",
    "MS Office",
    "Microsoft office",
    "web dev"
    
    
]
lang_list.extend(extra_skills)

In [99]:
exploded = []
for index, row in df.iterrows():
    #row = row.to_frame()
    details = row.details
    for skill in extra_skills:
        skill = skill.lower()
        if skill in details.lower():
            row["skill"] = skill
            row_temp = row.to_frame().transpose()
            exploded.append(row_temp)


In [100]:
results = pd.concat(exploded)

In [101]:
results.skill = results.skill.replace("r,", "R").replace(" r ", "R").replace("/r", "R").replace("r/", "R")
results.skill = results.skill.replace("c,", "C").replace(" c ", "C").replace("/c", "C").replace("c/", "C")
results = results.drop_duplicates()

In [102]:
results.head()

,companyName,date,details,rating,summary,title,url,location,skill
0,Robert Half,30+ days ago,2-7 years of experience\nExpert in SQL queries...,3.9,2-7 years of experience.\nExpert in SQL querie...,Data Scientist,https://www.indeed.com/viewjob?jk=7e25d46b2b2f...,NaN,python
0,Robert Half,30+ days ago,2-7 years of experience\nExpert in SQL queries...,3.9,2-7 years of experience.\nExpert in SQL querie...,Data Scientist,https://www.indeed.com/viewjob?jk=7e25d46b2b2f...,NaN,sql
0,Robert Half,30+ days ago,2-7 years of experience\nExpert in SQL queries...,3.9,2-7 years of experience.\nExpert in SQL querie...,Data Scientist,https://www.indeed.com/viewjob?jk=7e25d46b2b2f...,NaN,R
1,Microsoft,30+ days ago,Data Scientists at Microsoft help to improve t...,4.2,You will use data exploration techniques to di...,Full Time Opportunities for PhD Students or Re...,https://www.indeed.com/viewjob?jk=97ba8c5ac004...,"Redmond, WA 98052 (Overlake area)",statistics
1,Microsoft,30+ days ago,Data Scientists at Microsoft help to improve t...,4.2,You will use data exploration techniques to di...,Full Time Opportunities for PhD Students or Re...,https://www.indeed.com/viewjob?jk=97ba8c5ac004...,"Redmond, WA 98052 (Overlake area)",machine learning


In [103]:
results.shape

(1159, 9)

In [104]:
results.skill.value_counts().head(40)

R                                126
python                           101
machine learning                  80
computer science                  74
statistics                        66
sql                               64
excel                             61
aws                               46
mathematics                       45
ml                                44
spark                             37
C                                 31
database                          31
big data                          31
data engineer                     27
java                              26
deep learning                     25
c++                               25
git                               24
hadoop                            24
tensorflow                        21
data visualization                19
time series                       13
data engineering                  13
nlp                               12
c#                                10
natural language processing       10
s